## Пример парсинга нескольких страниц сайта госзакупок

Места, которые можно доработать, отмечены комментариями, начинающимися с TODO

!pip install bs4 requests

In [ ]:
import re
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

https://zakupki.gov.ru/epz/order/extendedsearch/results.html?searchString=&morphology=on&search-filter=Дате+обновления&pageNumber=2&sortDirection=false&recordsPerPage=_10&showLotsInfoHidden=false&savedSearchSettingsIdHidden=&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&placingWayList=&selectedLaws=&priceFromGeneral=&priceFromGWS=&priceFromUnitGWS=&priceToGeneral=&priceToGWS=&priceToUnitGWS=&currencyIdGeneral=-1&publishDateFrom=&publishDateTo=&applSubmissionCloseDateFrom=&applSubmissionCloseDateTo=&customerIdOrg=&customerFz94id=&customerTitle=&okpd2Ids=&okpd2IdsCodes=

In [ ]:
# Создаем список URL
URL = r"https://zakupki.gov.ru/epz/order/extendedsearch/results.html?"

# TODO: Изменить номера страниц
FIRST_PAGE = 2
LAST_PAGE = 4

urls_list = [] # создаем пустой список
for page in range(FIRST_PAGE, LAST_PAGE + 1): # перебираем страницы 
    
    # TODO: Варьировать другие параметры, использовать все параметры
    page_url = f'{URL}searchString=&morphology=on&search-filter=Дате+обновления&pageNumber={page}&sortDirection=false&recordsPerPage=_10&showLotsInfoHidden=false&savedSearchSettingsIdHidden=&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&placingWayList=&selectedLaws=&priceFromGeneral=&priceFromGWS=&priceFromUnitGWS=&priceToGeneral=&priceToGWS=&priceToUnitGWS=&currencyIdGeneral=-1&publishDateFrom=&publishDateTo=&applSubmissionCloseDateFrom=&applSubmissionCloseDateTo=&customerIdOrg=&customerFz94id=&customerTitle=&okpd2Ids=&okpd2IdsCodes='
    urls_list.append(page_url) # записываем адреса в список
    
print('Длина полученного списка веб-страниц: {}'.format(len(urls_list)))
print('Пример записанных в список страниц: {}'.format(urls_list))

In [ ]:
type(urls_list[0])

In [ ]:
page = requests.get(urls_list[0]) # попробуем спарсить одну страницу
print(page.status_code) # если 200, то успешно подключились к странице

In [ ]:
urls_list[0]

In [ ]:
requests.get(urls_list[1]) 

In [ ]:
page

In [ ]:
page.text

In [ ]:
type(page.text)

In [ ]:
soup = BeautifulSoup(page.text, "html.parser") # посмотрим как выглядит спарсенная страница
soup

In [ ]:
type(soup)

In [ ]:
soup.find_all('div', class_='registry-entry__header-mid__number')

In [ ]:
soup.find_all('div', class_='registry-entry__header-mid__number')[0]

In [ ]:
soup.find_all('div', class_='registry-entry__header-mid__number')[0].text

In [ ]:
%%time

numbers_list = [] 
placements_dates_list = []
update_dates_list = []
finish_dates_list = []
objects_list = []
prices_list = []

procurements_status_dict = {}

for i, url in enumerate(urls_list):
    print(url, end=" ")
    # TODO: добавить try-except и цикл, чтобы несколько раз пытаться подключиться
    page = requests.get(url) # отправляем запрос
    print(page.status_code)
    
    procurements_status_dict[url] = page.status_code
    
    if page.status_code == 200: # если ссылка рабочая, код = 200
        soup = BeautifulSoup(page.text, 'html.parser')
    else:
        print("Не удалось подключиться", page.status_code)
        continue
        
    # Ищем номера закупок
    numbers = soup.find_all('div', class_='registry-entry__header-mid__number') # ищем номер заявки
    if numbers:
        numbers_list.append([number.text for number in numbers])
    else:
        numbers_list.append(np.nan)
        
    
    # TODO: Добавить проверки, что у закупки есть все три даты 
    # TODO: Например, пропускать те закупки, где нет всех трех дат, проверку делать отдельно до сохранения в списки
    dates = soup.find_all('div', class_='data-block__value') # ищем дату размещения
    if dates:
        placements_dates_list.append([dates[i].text for i in range(0, len(dates), 3)])
        update_dates_list.append([dates[i].text for i in range(1, len(dates), 3)])
        finish_dates_list.append([dates[i].text for i in range(2, len(dates), 3)])
    else:
        placements_dates_list.append(np.nan)
        update_dates_list.append(np.nan)
        finish_dates_list.append(np.nan)
        
    
    objects = soup.find_all('div', class_='registry-entry__body-value') # ищем объекты закупок
    if objects:
        objects_list.append([object_.text for object_ in objects])
    else:
        objects_list.append(np.nan)
    
    prices = soup.find_all('div', class_='price-block__value') # ищем цены закупок
    if prices:
        prices_list.append([price.text for price in prices])
    else:
        prices_list.append(np.nan)

    # TODO: менять паузу
    time.sleep(2.0) # пауза для обхода блокировки 

In [ ]:
import random

random.random() * 2

In [ ]:
procurements_status_dict

In [ ]:
prices_list

In [ ]:
prices_list[0][0]

In [ ]:
type(prices_list[0][0])

In [ ]:
price_ = re.sub("[^0-9,₽]", "", prices_list[0][0])

In [ ]:
type(price_)

In [ ]:
price_

In [ ]:
float(price_)

In [ ]:
price_ = re.sub("[^0-9,]", "", prices_list[0][0])

In [ ]:
price_

In [ ]:
type(price_)

In [ ]:
price_.replace(',', '.')

In [ ]:
float(price_.replace(',', '.'))

In [ ]:
price_float = float(price_.replace(',', '.'))

In [ ]:
type(price_float)

In [ ]:
price_float

In [ ]:
currency = prices_list[0][0].split()[-1]

In [ ]:
currency

In [ ]:
prices_list[0]

In [ ]:
len(numbers_list[0])

In [ ]:
len(objects_list[0])

---

In [ ]:
numbers_list[0][0]

In [ ]:
placements_dates_list[0][0]

In [ ]:
update_dates_list[0][0]

In [ ]:
objects_list[0][0]

In [ ]:
prices_list[0][0]

In [ ]:
procurements = []

# Проходим по всем страницам
for numbers, placements_dates, update_dates, finish_dates, objects_, prices in zip(numbers_list, 
                                                                                    placements_dates_list,
                                                                                    update_dates_list, 
                                                                                    finish_dates_list, 
                                                                                    objects_list, 
                                                                                    prices_list):
    # Проходим по всем закупкам
    for number, placements_date, update_date, finish_date, object_, price in zip(numbers, 
                                                                                 placements_dates, 
                                                                                 update_dates, 
                                                                                 finish_dates, 
                                                                                 objects_, 
                                                                                 prices ):
        # TODO: сохранять валюту (единица измерения)                                                                       
        procurements.append({
            "Номер закупки": re.sub("[^0-9]", "", number),
            "Дата размещения": placements_date,
            "Дата обновления": update_date,
            "Дата окончания": finish_date,
            "Объект закупки": object_,
            "Начальная цена": re.sub("[^0-9,]", "", price)
        })

In [ ]:
procurements

In [ ]:
df = pd.DataFrame(procurements)
df.head()

In [ ]:
df.to_csv("gov2.csv", encoding="utf-8")